# Transparent Naive Bayes

## Overview

This project is meant to serve as an educational tool for me and anyone else who wants to understand how Naive Bayes classifiers are implemented in code. My goal is to clearly document each step of the algorithm, including the underlying math as much as possible. Furthermore, I will try to keep the code as simple and easy-to-understand as possible, sacrificing performance and robustness if necessary.

## Bernoulli Naive Bayes

We'll first start with an implementation of Bernoulli Naive Bayes. Bernoulli Naive Bayes is the simplest form of Naive Bayes classification because it assumes all features and labels are Bernoulli trails, i.e. binary values. If each feature in the input represents an event, then a 1 means the event happened, and a 0 means it did not.

Let's say we're trying to predict whether we'll have rain given current weather conditions. The features we have to work with are clouds, wind, humidity, and thunder. Note that these are all _events_, so humidity isn't a measurement -- it's either humid or not. A slice of our dataset may look something like this:

<table>
    <tr>
        <th>Clouds?</th>
        <th>Wind?</th>
        <th>Humid?</th>
        <th>Thunder?</th>
        <th>Rain?</th>
    </tr>
    <tr>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
    </tr>
    <tr>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
    </tr>
    <tr>
        <td>1</td>
        <td>1</td>
        <td>1</td>
        <td>1</td>
        <td>1</td>
    </tr>
</table>

We're trying to predict whether it will rain given these conditions, so in mathematical terms, that's $P(rain \mathbin{\vert} clouds,wind,humid,thunder)$. Using Bayes' Rule, we get:

$$ P(rain \mathbin{\vert} clouds,wind,humid,thunder) = \frac{P(clouds,wind,humid,thunder \mathbin{\vert} rain)P(rain)}{P(clouds,wind,humid,thunder)} $$

Note that we're just trying to predict an outcome (1 or 0) for a given sample of weather conditions -- we don't care about the actual _probability_. In other words, if our model predicts a 70% chance of rain for one sample and a 55% chance of rain for another sample, both samples will result in an output of 1 -- _that it most likely will rain_. Therefore, we don't have to worry about the denominator in the above equation.

Furthermore, naive bayes assumes __conditional independence__, meaning that it doesn't care about the chance of any event occuring based on other events. Obviously, in the real world, the occurrence of events often depends on other events happening. We don't expect thunder to occur on a sunny day. However, this assumption allows us to do a neat trick that makes computation a lot easier, and it still works pretty well on real data! The equation below shows what we're actually trying to solve.

$$ argmax_{rain}P(rain \mathbin{\vert} clouds,wind,humid,thunder) = P(clouds \mathbin{\vert} rain)P(wind \mathbin{\vert} rain)P(humid \mathbin{\vert} rain)P(thunder \mathbin{\vert} rain)P(rain) $$

Each of the values on the right side of the above equation can be calculated with the priors we have from the dataset. For example, to calculate $P(thunder \mathbin{\vert} rain)$ we count the number of rows with thunder AND rain, and divide by the number of rows with rain to get $0.5$.

### Building a Dataset

Let's build a dataset based off this example. We'll randomly generate the weather conditions, and then determine if rain happened based on those conditions with a little noise thrown in.

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

np.random.seed(42)

X = np.random.binomial(1, 0.5, size=(1000, 3))

# There's only a 20% chance of thunder if it's cloudy in this world
thunder_chance = 0.2
thunder = []
for x in X:
    if x[0] == 1 and np.random.random() < thunder_chance:
        thunder.append(1)
    else:
        thunder.append(0)

thunder = np.array([thunder]).reshape((-1, 1))
X = np.append(X, thunder, axis=1)
y = []

for x in X:
    rain_chance = x[0] * (0.5 * x[0] + 0.05 * x[1] + 0.1 * x[2] + 0.3 * x[3])
    output = 1 if np.random.random() < rain_chance else 0
    y.append(output)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df = pd.DataFrame(X, columns=['cloudy', 'windy', 'humid', 'thunder'])
df['rain'] = y

df.loc[:30]

,cloudy,windy,humid,thunder,rain
0,0,1,1,0,0
1,1,0,0,0,1
2,0,1,1,0,0
3,1,0,1,0,1
4,1,0,0,0,1
5,0,0,1,0,0
6,0,0,1,0,0
7,0,0,0,0,0
8,0,1,0,0,0
9,1,1,0,0,1


In [19]:
from sklearn.naive_bayes import BernoulliNB as SKLearnBernoulliNB
from sklearn.metrics import accuracy_score
from bernoulli_nb import BernoulliNB

def evaluate_model(model_class, X_train, X_test, y_train, y_test, params={}, name=None):
    model = model_class(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    
    if name is None:
        name = model.__class__.__name__
    print("Accuracy score of {}: {:.2f}%".format(name, accuracy_score(y_test, pred) * 100))

evaluate_model(SKLearnBernoulliNB, X_train, X_test, y_train, y_test, {'alpha': 0, 'binarize': None}, 'Benchmark')
evaluate_model(BernoulliNB, X_train, X_test, y_train, y_test)

Accuracy score of Benchmark: 84.00%
Accuracy score of BernoulliNB: 84.00%


C:\Users\vgmd9_000\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\vgmd9_000\Software_Development\machine-learning\projects\transparent_bayes\bernoulli_nb.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  f_count = len(X.loc[y_true][X[f] == 1])
C:\Users\vgmd9_000\Software_Development\machine-learning\projects\transparent_bayes\bernoulli_nb.py:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  f_count = len(X.loc[y_false][X[f] == 1])


### Dataset

The dataset I will be using is the [Credit Approval Dataset](http://archive.ics.uci.edu/ml/datasets/Credit+Approval) from the UCI Machine Learning Repository. It has a good mix of continuous and categorical attributes, and a binary label.

### Data Exploration

Now I will load the data into a pandas dataframe and view its statistics.

In [ ]:
import pandas as pd
import numpy as np

names = []
for i in range(1, 16):
    names.append("A" + str(i))
names.append("approve?")

dtype = {'A1': str,
         'A2': np.float32,
         'A3': np.float32,
         'A4': str,
         'A5': str,
         'A6': str,
         'A7': str,
         'A8': np.float32,
         'A9': str,
         'A10': str,
         'A11': np.float32,
         'A12': str,
         'A13': str,
         'A14': np.float32,
         'A15': np.float32,
         'approve?': str}

data = pd.read_csv("./data.csv", header=None, names=names, dtype=dtype, na_values=['?'])

print(data.head())
print(data.describe())

### Data Preprocessing

To preprocess the data, we'll first drop the rows with NaN values, then remove the labels from the dataset. We then one-hot encode the categorical columns, and normalize the entire dataset using min-max scaling.

In [ ]:
data.dropna(axis=0, inplace=True)

y = data['approve?']
X = data.drop('approve?', axis=1)

X = pd.get_dummies(X)

X = (X - X.min()) / (X.max() - X.min())

X.head()

### Splitting the data

Now we split the data into training and testing sets. We use a random state for reproducible results. Because we're only comparing our model against the benchmark, we don't need to go to the extent of implementing K-Fold cross validation.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Benchmark Model

We will use scikit-learn's GaussianNB as the benchmark to test our from-scratch model against.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

model = GaussianNB()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("Accuracy: {:.2f}%".format(accuracy_score(y_test, pred) * 100))